In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import pickle
from networks import *
from fbsde import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
h = 1
epsilon = 0.01

In [3]:
max_level = 6
T = 1
r = 0.05
volatility = 0.4
batch_size = 100
iterations_per_level = 1000
fbsde = BS_Barenblatt(volatility, r, T)
learning_rate = 1e-3
runs = 10
losses = []
terminal_losses = []
Y0s = []
times = []

In [4]:
for d in [100, 3]:
    zeta = torch.tensor(int(d / 2) * [1., 0.5] + (d % 2) * [1.], device=device)
    for run in range(runs):
        run_losses = []
        run_terminal_losses = []
        run_times = []
        run_Y0s = []
        network = NAIS_Net_Untied(d + 1, 256, 4, 1, torch.sin, epsilon, h).to(device) 
        optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
        N = 2
        for level in range(max_level):
            step_size = fbsde.T / N
            for iteration in range(iterations_per_level):
                start_time = time.time()
                loss = 0
                t0 = 0
                X0 = zeta.repeat(batch_size, 1).view(batch_size, d).requires_grad_(True)  # M x D
                t0_X0 = torch.cat((t0 * torch.ones((batch_size, 1), device=device), X0), dim=1)
                Y0 = network(t0_X0)
                Z0 = torch.autograd.grad(Y0, X0, grad_outputs=torch.ones_like(Y0), create_graph=True)[0]
                mean_Y_init = torch.mean(Y0)
                for i in range(N):
                    t1 = t0 + step_size
                    delta_W = torch.randn((batch_size, d), device=device) * np.sqrt(step_size)
                    X1 = X0 + fbsde.mu(t0, X0, Y0, Z0) * (t1 - t0) + (delta_W.unsqueeze(-2) @ fbsde.sigma(t0, X0, Y0)).squeeze(-2)
                    #Y1_tilde = Y0 + fbsde.phi(t0, X0, Y0, Z0) * (t1 - t0) + torch.sum((delta_W.unsqueeze(-2) @ fbsde.sigma(t0, X0, Y0)).squeeze(-2) * Z0, dim=1, keepdim=True)
                    t1_X1 = torch.cat((t1 * torch.ones((batch_size, 1), device=device), X1), dim=1)
                    Y1 = network(t1_X1)
                    Z1 = torch.autograd.grad(Y1, X1, grad_outputs=torch.ones_like(Y1), create_graph=True)[0]
                    #loss += torch.sum((Y1 - Y1_tilde) ** 2)
                    loss += torch.sum((Y1 - Y0 - fbsde.phi(t0, X0, Y0, Z0) * (t1 - t0) - torch.sum((delta_W.unsqueeze(-2) @ fbsde.sigma(t0, X0, Y0)).squeeze(-2) * Z0, dim=1, keepdim=True)) ** 2)
                    t0 = t1
                    X0 = X1
                    Y0 = Y1
                    Z0 = Z1
                    #X.append(X0)
                    #Y.append(Y0)
                terminal_loss = torch.sum(torch.pow(Y1 - fbsde.g(X1), 2))
                loss += terminal_loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                run_losses.append(loss.item())
                run_terminal_losses.append(terminal_loss.item())
                run_Y0s.append(mean_Y_init.item())
                run_times.append(time.time() - start_time)
                if iteration % 100 == 0:
                    print("Run: %d, Iteration: %d, Loss: %.3e, Y0: %.3f, Terminal Loss: %3e" % (run, iteration, loss.item(), mean_Y_init, terminal_loss))
                    torch.save(network.state_dict(), "FBSDE_Euler/fbsde_euler_dimensions_%d_run_%d_level_%d_iteration_%d.pt" % (d, run, level, iteration))
            N *= 2
        torch.save(network.state_dict(), "FBSDE_Euler/fbsde_euler_dimensions_%d_run_%d_trained.pt" % (d, run))
        losses.append(run_losses)
        terminal_losses.append(run_terminal_losses)
        Y0s.append(run_Y0s)
        times.append(run_times)
with open("FBSDE_Euler/losses.pkl", "wb") as f:
    pickle.dump(losses, f)
with open("FBSDE_Euler/terminal_losses.pkl", "wb") as f:
    pickle.dump(terminal_losses, f)
with open("FBSDE_Euler/times.pkl", "wb") as f:
    pickle.dump(run_times, f)
with open("FBSDE_Euler/Y0s.pkl", "wb") as f:
    pickle.dump(Y0s, f)

Run: 0, Iteration: 0, Loss: 5.113e+05, Y0: -0.143, Terminal Loss: 5.113269e+05
Run: 0, Iteration: 100, Loss: 2.790e+03, Y0: 73.199, Terminal Loss: 2.718803e+03
Run: 0, Iteration: 200, Loss: 1.975e+03, Y0: 72.565, Terminal Loss: 1.963538e+03
Run: 0, Iteration: 300, Loss: 2.345e+03, Y0: 71.560, Terminal Loss: 2.341008e+03
Run: 0, Iteration: 400, Loss: 1.114e+03, Y0: 73.696, Terminal Loss: 1.094095e+03
Run: 0, Iteration: 500, Loss: 8.346e+02, Y0: 73.936, Terminal Loss: 7.677119e+02
Run: 0, Iteration: 600, Loss: 4.587e+02, Y0: 75.520, Terminal Loss: 4.319131e+02
Run: 0, Iteration: 700, Loss: 4.750e+02, Y0: 75.812, Terminal Loss: 4.311903e+02
Run: 0, Iteration: 800, Loss: 3.605e+02, Y0: 76.378, Terminal Loss: 3.488679e+02
Run: 0, Iteration: 900, Loss: 4.047e+02, Y0: 77.133, Terminal Loss: 3.945793e+02
Run: 0, Iteration: 0, Loss: 3.849e+02, Y0: 76.347, Terminal Loss: 3.799175e+02
Run: 0, Iteration: 100, Loss: 5.689e+02, Y0: 75.957, Terminal Loss: 5.637540e+02
Run: 0, Iteration: 200, Loss: 5.

In [6]:
losses_100 = losses[:10]
terminal_losses_100 = terminal_losses[:10]
Y0s_100 = Y0s[:10]
times_100 = times[:10]
with open("FBSDE_Euler/losses_dimensions_100.pkl", "wb") as f:
    pickle.dump(losses_100, f)
with open("FBSDE_Euler/terminal_losses_dimensions_100.pkl", "wb") as f:
    pickle.dump(terminal_losses_100, f)
with open("FBSDE_Euler/times_dimensions_100.pkl", "wb") as f:
    pickle.dump(times_100, f)
with open("FBSDE_Euler/Y0s_dimensions_100.pkl", "wb") as f:
    pickle.dump(Y0s_100, f)
losses_3 = losses[10:]
terminal_losses_3 = terminal_losses[10:]
Y0s_3 = Y0s[10:]
times_3 = times[10:]
with open("FBSDE_Euler/losses_dimensions_3.pkl", "wb") as f:
    pickle.dump(losses_3, f)
with open("FBSDE_Euler/terminal_losses_dimensions_3.pkl", "wb") as f:
    pickle.dump(terminal_losses_3, f)
with open("FBSDE_Euler/times_dimensions_3.pkl", "wb") as f:
    pickle.dump(times_3, f)
with open("FBSDE_Euler/Y0s_dimensions_3.pkl", "wb") as f:
    pickle.dump(Y0s_3, f)

In [2]:
Y0s_100 = Y0s[:10]
with open("FBSDE_Euler/Y0s_dimensions_100.pkl", "wb") as f:
    pickle.dump(Y0s_100, f)

NameError: name 'Y0s' is not defined

In [3]:
f = open("FBSDE_Euler/Y0s_dimensions_100.pkl", "rb")
Y0s = pickle.load(f)
f.close()

In [4]:
Y0s.sh

[[-0.14304664731025696,
  2.887003183364868,
  5.880185604095459,
  8.840619087219238,
  11.797211647033691,
  14.777355194091797,
  17.79319190979004,
  20.842350006103516,
  23.899076461791992,
  26.934946060180664,
  29.934650421142578,
  32.885128021240234,
  35.77909469604492,
  38.61240768432617,
  41.375770568847656,
  44.05097579956055,
  46.62217330932617,
  49.09171676635742,
  51.445457458496094,
  53.6781005859375,
  55.77736282348633,
  57.74247741699219,
  59.58100509643555,
  61.293304443359375,
  62.88794708251953,
  64.35204315185547,
  65.70560455322266,
  66.95079040527344,
  68.09765625,
  69.13772583007812,
  70.08357238769531,
  70.939208984375,
  71.71144104003906,
  72.38695526123047,
  72.96713256835938,
  73.44278717041016,
  73.77896881103516,
  73.9666748046875,
  73.93694305419922,
  73.61466979980469,
  73.03984069824219,
  72.23993682861328,
  71.28140258789062,
  70.74571228027344,
  70.60902404785156,
  70.7847900390625,
  71.17254638671875,
  71.758033